# Reflection Gates with multiplexor multi-controlled Z gates.
In the **01_Multi_Controlled_Gate_Decomposition.ipynb** notebook hte different functions for creating multi-controlled pahse (and Z) gates were explained.  In this notebook we are going to use this functions for creating the reflection gates mandatory for the Grover-like operator

In [ ]:
import sys
sys.path.append("../../../")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [ ]:
#This cell loads the QLM solver.
#QLMaaS == False -> uses PyLinalg
#QLMaaS == True -> try to use LinAlg (for using QPU as CESGA QLM one)
from QQuantLib.utils.qlm_solver import get_qpu
QLMaaS = False
linalg_qpu = get_qpu(QLMaaS)

In [ ]:
#See 01_DataLoading_Module_Use for the use of this function
from QQuantLib.utils.data_extracting import get_results

In [ ]:
from QQuantLib.DL.data_loading import uniform_distribution
#Testing Function. Apply a Uniform distribution and then an input gate
def testing_gate(input_gate):
    """
    Function for testing purpouses. Given a QLM gate creates a uniform distribution based
    on the arity if the inbput gate and apply the gate.
    
    Parameters
    ----------
    
    input_gate : QLM routine 
        QLM routine user want to test
        
    Returns
    _______
    
    routine : QLM routine 
        QLM routine for testing input gate. 
    
    
    """
    number_qubits = input_gate.arity
    routine = qlm.QRoutine()
    register = routine.new_wires(number_qubits)    
    routine.apply(uniform_distribution(number_qubits), register)
    routine.apply(input_gate, register)
    return routine

## 1. Multiplexors functions in library.

We have stored following functions from notebook in module **QQuantLib.AA.amplitude_amplification**:
* **phase_multiplexor_base**
* **recursive_multiplexor**
* **multiplexor_controlled_ph**
* **multiplexor_controlled_z**

In this section we, briefly, test the use of the two necesary functions  *multiplexor_controlled_ph* and  *multiplexor_controlled_z*

In [ ]:
from QQuantLib.AA.amplitude_amplification import multiplexor_controlled_ph, multiplexor_controlled_z

### 1.1 Multi-Controlled Phase Gate

In [ ]:
number_of_qbits = 4
phase_angle = np.pi/4.0
mc_ph = multiplexor_controlled_ph(phase_angle, number_of_qbits)
print('Multi-Controlled Phase with Multiplexors')
%qatdisplay mc_ph --depth 0
test_mc_ph = testing_gate(mc_ph)
print("Testing Multi-Controlled Phase with Multiplexors")
%qatdisplay test_mc_ph
results_mc_ph, _, _, _ = get_results(test_mc_ph, linalg_qpu=linalg_qpu, shots=0)

In [ ]:
#·Comparison  with QLM implementation
c_phase_qlm = qlm.PH(phase_angle).ctrl(number_of_qbits-1)
print("QLM Controlled Phase Gate")
%qatdisplay c_phase_qlm
test_c_phase_qlm = testing_gate(c_phase_qlm)
print("Testing Cricuit for QLM Controlled Phase Gate")
%qatdisplay test_c_phase_qlm
results_c_phase_qlm, _, _, _ = get_results(test_c_phase_qlm, linalg_qpu=linalg_qpu, shots=0)

In [ ]:
Testing_columns = ['Int_lsb', 'Probability', 'Amplitude']
np.isclose(results_mc_ph[Testing_columns], results_c_phase_qlm[Testing_columns]).all()

### 1.2 Multi-Controlled Z Gate

In [ ]:
number_of_qbits = 3
phase_angle = np.pi/4.0
mc_z = multiplexor_controlled_z(number_of_qbits)
print('Multi-Controlled Z with Multiplexors')
%qatdisplay mc_z --depth 
test_mc_z = testing_gate(mc_z)
print("Testing Multi-Controlled Z with Multiplexors")
%qatdisplay test_mc_ph --depth
results_mc_z, c, _, _ = get_results(test_mc_z, linalg_qpu=linalg_qpu, shots=0)

In [ ]:
c.statistics()

In [ ]:
#·Comparison  with QLM implementation
c_Z_qlm = qlm.Z.ctrl(number_of_qbits-1)
print("QLM Controlled Z Gate")
%qatdisplay c_Z_qlm
test_c_Z_qlm = testing_gate(c_Z_qlm)
print("Testing Cricuit for QLM Controlled Phase Gate")
%qatdisplay test_c_Z_qlm
results_c_Z_qlm, _, _, _ = get_results(test_c_Z_qlm, linalg_qpu=linalg_qpu, shots=0)

In [ ]:
Testing_columns = ['Int_lsb', 'Probability', 'Amplitude']
np.isclose(results_mc_z[Testing_columns], results_c_Z_qlm[Testing_columns]).all()

## 2. Testing Reflection

In [ ]:
from QQuantLib.DL.data_loading import load_probability, load_array

Creating staff for loading data

In [ ]:
n = 3
N = 2**n
x = np.arange(N)
p = x/np.sum(x)
probability_routine = qlm.QRoutine()
register = probability_routine.new_wires(n)
probability_routine.apply(load_probability(p),register)
%qatdisplay probability_routine --depth 1 --svg

In [ ]:
from QQuantLib.AA.amplitude_amplification import reflection

In [ ]:
reflection1 = qlm.QRoutine()
reflection1_register = reflection1.new_wires(n)
reflection1.apply(probability_routine,reflection1_register)
#Step 1: Creating reflection QLM Abstract Gate
ReflectionGate = reflection([1,1,1], mcz_qlm=False)
%qatdisplay ReflectionGate --depth 0 --svg
#Step 2: Applying ReflectionGate to the affected qbits: In present case the reflection gate affects all the qbits of the system
reflection1.apply(ReflectionGate, reflection1_register)
%qatdisplay reflection1 --depth 1 --svg

In [ ]:
results_reflection1, _, _, _ = get_results(reflection1, linalg_qpu=linalg_qpu, shots=0)
amplitudes_reflection1 = results_reflection1["Amplitude"].values
results_reflection1

In [ ]:
print("Classical amplitudes: ", np.sqrt(p))
print("Quantum amplitudes: ",amplitudes_reflection1)

In [ ]:
print('Test OK: ',
    np.isclose(np.sqrt(p)[:-1], amplitudes_reflection1[:-1]).all() and
    np.isclose(np.sqrt(p)[-1], -amplitudes_reflection1[-1]).all()
     )